We need to install the leafmap client for data visualization in the notebook.

**Note:** This is set to run silently so you will not see an output when executing this cell. If you'd like to ensure the package downloads successfully remove the `--quiet` flag

In [ ]:
! pip install leafmap numpy --quiet

Import the python libraries required for running the script

In [45]:
import json
import leafmap
import numpy as np
import xarray as xr

This is a dictionary contains several sample cases of burn scars across the globe. These configuration settings will automatically set the location of the map for AOI selection and the start_date and end_date to perform the data query.

In [46]:
#configure settings for selected events
INFERENCE_URL = 'https://hls-01.workshop.nasa-impact.net'

EVENT_DETAILS = {
    'mongolian_fire': {
        'center_lon': 119.3,
        'center_lat': 47.1,
        'default_zoom': 8,
        'start_date': '2022-04-19T00:00:00Z',
        'end_date': '2022-04-19T23:59:59Z'
    },
    'new_mexico_black_fire': {
        'center_lon': -107.5,
        'center_lat': 33.5,
        'default_zoom': 10,
        'start_date': '2022-05-16T00:00:00Z',
        'end_date': '2022-06-10T23:59:59Z'
    },
    'alberta_fire': {
        'center_lon': -124.2,
        'center_lat': 61.8,
        'default_zoom': 8,
        'start_date': '2023-05-27T00:00:00Z',
        'end_date': '2023-05-28T23:59:59Z'
    },
    'maui_fire': {
        'center_lon': -156.659394,
        'center_lat': 20.886984,
        'default_zoom': 12,
        'start_date': '2023-08-13T00:00:00Z',
        'end_date': '2023-08-13T23:59:59Z'
    }
}

Select the predefined event of your choice from above. If you'd like to execute a different event, the simplest way to implement it would be to add the event details as a new key in the dictionary

In [47]:
event = 'maui_fire'
event_details = EVENT_DETAILS[event]

In [48]:
datestring = event_details['start_date']
HLSL30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_L30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'
HLSS30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_S30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'

Initialize map with one of the canned examples listed above, and HLS XYZ Layers from NASA Worldview. If you would like execute a custom use case, please navigate to your desired AOI, draw a bounding box, and update the start_date and end_date in the prepare_items method

In [49]:
map = leafmap.Map(
        center=(event_details['center_lat'],
        event_details['center_lon']), 
        zoom=event_details['default_zoom'], 
        draw_control=True, 
        measure_control=False, 
        fullscreen_control=False, 
        attribution_control=True
    )
map.add_tile_layer(HLSL30_TILE_LAYER, name='HLSL30', attribution='NASA')
map.add_tile_layer(HLSS30_TILE_LAYER, name='HLSS30', attribution='NASA')
map

Map(center=[20.886984, -156.659394], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

This cell saves the bounding box drawn by the user.

**WARNING:** if you skip the next step, the notebook will not find any HLS data as there will be no bounding box information.

In [50]:
# Save the selected location as geojson
map.save_draw_features("bbox.geojson")

**WARNING:** if you skipped the previous step, the notebook will not find any HLS data as there will be no bounding box information.

In [51]:
# Read the bounding box and print
bbox = json.load(open("bbox.geojson", "r"))
bbox

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-156.698341, 20.830898],
      [-156.698341, 20.917831],
      [-156.635857, 20.917831],
      [-156.635857, 20.830898],
      [-156.698341, 20.830898]]]}}]}

We will utilize the trained model, changed configuration file, and the date information to get a prediction on the selected region.

In [52]:

def bbox_from_geojson(geojson):
    """
        Get the coordinates of bounding box from an extended notation to flat coordinate
        notation
    Args:
        geojson: File path of geojson

    Returns:
        list: [left, down, right, top]
    """
    with open(geojson) as geojson_file:
        geojson_detail = json.load(geojson_file)
    coordinates = np.asarray(geojson_detail['features'][0]['geometry']['coordinates'])
    lats = coordinates[:, :, 1]
    lons = coordinates[:, :, 0]
    return [lons.min(), lats.min(), lons.max(), lats.max()]

In [53]:
# Convert geojson to flat bounding box representation.
bbox = bbox_from_geojson('bbox.geojson')
bbox

[-156.698341, 20.830898, -156.635857, 20.917831]

Note: `BUCKET_NAME`, `new_config_filename`, and `MODEL_NAME` are variables set in the previous notebook. Please copy paste those variables here for this step to run smoothly.


In [54]:
import requests
import json

# prepare payload

payload = json.dumps({
  "config_path": f"s3://{BUCKET_NAME}/data/configs/{new_config_filename}",
  "model_path": f"s3://smd-workshop-01/{MODEL_NAME}",
  "model_type": "burn_scars",
  "date": event_details['start_date'].split('T')[0],
  "bounding_box": bbox
})

headers = {
  'Content-Type': 'application/json'
}

# Use deployed app to get inference on the selected date/location
response = requests.request(
      "POST", 
      f"{INFERENCE_URL}/infer", 
      headers=headers, 
      data=payload
  )

predictions = response.json()

predictions

{'predictions': {'type': 'FeatureCollection',
  'features': [{'id': '0',
    'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': [[[-156.67799949646272, 20.90780930177973],
       [-156.67782783508636, 20.907648943216298],
       [-156.67714118957213, 20.907648943216298],
       [-156.67696952819583, 20.907488584481424],
       [-156.6766262054432, 20.907488584481424],
       [-156.6764545440669, 20.907328225575107],
       [-156.67628288269057, 20.907328225575107],
       [-156.67576789856165, 20.906847147827435],
       [-156.67559623718532, 20.906847147827435],
       [-156.67439460754215, 20.905724627073898],
       [-156.67439460754215, 20.905564266281687],
       [-156.67405128478953, 20.905243544182937],
       [-156.67405128478953, 20.905083182876407],
       [-156.6735363006606, 20.904602097928212],
       [-156.6735363006606, 20.904441735935954],
       [-156.67302131653167, 20.90396064893059],
       [-156.67302131653167, 20.90380

In [55]:
geojson = predictions['predictions']

detection_map = leafmap.Map(
        center=(event_details['center_lat'],
        event_details['center_lon']), 
        zoom=event_details['default_zoom'], 
        draw_control=True, 
        measure_control=False, 
        fullscreen_control=False, 
        attribution_control=True
    )
detection_map.add_tile_layer(HLSL30_TILE_LAYER, name='HLSL30', attribution='NASA')
detection_map.add_tile_layer(HLSS30_TILE_LAYER, name='HLSS30', attribution='NASA')
detection_map.add_geojson(geojson, layer_name=f"{event}-detections")

detection_map

Map(center=[20.886984, -156.659394], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…